In [1]:
from os.path import join
from nipype.interfaces.fsl import Threshold, ImageStats
from nipype.interfaces.freesurfer.preprocess import MRIConvert
import nibabel as nb
import pandas as pd
from glob import glob
from datetime import date

In [8]:
#home = '/gpfs/milgram/project/gee_dylan/candlab'
home = '/Users/lucindasisk/Desktop/Milgram/candlab'
dest = join(home, 'analyses/space/hipp_volume')
homedest = '/Users/lucindasisk/Box/LS_folders/CANDLab/Projects/SPACE_R61_HippVolumes'
dataV1 = join(home, 'data/mri/space_freesurfer/V1')
dataV2 = join(home, 'data/mri/space_freesurfer/V1')
subject_list = pd.read_csv('/Users/lucindasisk/Box/LS_folders/CANDLab/Projects/SPACE_R61_HippVolumes/IDs_R61.csv', header=0).iloc[:,0].astype(str)       
today = str(date.today())

In [4]:
#Left hippocampus segmentation from aparc+asegA2009s: 17
#Right hippocampus segmentation from aparc+asegA2009s: 53
hipp_volumes = []

def get_volume(subject, datapath):
    #Convert .mgz file
    mriconvert = MRIConvert(in_file = join(datapath, 'sub-'+subject, 'mri', 'aparc.a2009s+aseg.mgz'),
                           out_file = join(datapath, 'sub-'+subject, 'mri', 'aparc.a2009s+aseg.nii.gz'),
                           out_type='niigz').run()
    #Set filepath
    seg_img = join(datapath, 'sub-'+subject, 'mri', 'aparc.a2009s+aseg.nii.gz')
    #Extract left and right hippocampi (thresholding); left is 17, right is 53
    extract_hipp_left = Threshold(in_file=seg_img, thresh = 16.5, 
                          direction = 'below', 
                          args = '-uthr 17.5',
                         out_file = join(datapath, 'sub-'+subject, 'mri','left_hippocampus.nii.gz')).run()
    extract_hipp_right = Threshold(in_file=seg_img, thresh = 52.5, 
                          direction = 'below', 
                          args = '-uthr 53.5',
                         out_file = join(datapath, 'sub-'+subject, 'mri','right_hippocampus.nii.gz')).run()
    #Extract left and right amygdalae; left is 18, right is 54
    extract_amyg_left = Threshold(in_file=seg_img, thresh = 17.5, 
                          direction = 'below', 
                          args = '-uthr 18.5',
                         out_file = join(datapath, 'sub-'+subject, 'mri','left_amygdala.nii.gz')).run()
    extract_amyg_right = Threshold(in_file=seg_img, thresh = 53.5, 
                          direction = 'below', 
                          args = '-uthr 54.5',
                         out_file = join(datapath, 'sub-'+subject, 'mri','right_amygdala.nii.gz')).run()
    
    #Extract left and right cerebellum; left is 8, right is 47
    extract_cerebellum_left = Threshold(in_file=seg_img, thresh = 7.5, 
                          direction = 'below', 
                          args = '-uthr 8.5',
                         out_file = join(datapath, 'sub-'+subject, 'mri','left_cerebellum.nii.gz')).run()
    extract_cerebellum_right = Threshold(in_file=seg_img, thresh = 46.5, 
                          direction = 'below', 
                          args = '-uthr 47.5',
                         out_file = join(datapath, 'sub-'+subject, 'mri','right_cerebellum.nii.gz')).run()
    
    #Calculate number of voxels (volume)
    hipp_left_vox = ImageStats(in_file = join(datapath, 'sub-'+subject, 'mri','left_hippocampus.nii.gz'),
                           op_string = "-V").run()
    hipp_right_vox = ImageStats(in_file = join(datapath, 'sub-'+subject, 'mri','right_hippocampus.nii.gz'),
                            op_string = "-V").run()
    
    #Calculate number of amygdala voxels (volume)
    amyg_left_vox = ImageStats(in_file = join(datapath, 'sub-'+subject, 'mri','left_amygdala.nii.gz'),
                           op_string = "-V").run()
    amyg_right_vox = ImageStats(in_file = join(datapath, 'sub-'+subject, 'mri','right_amygdala.nii.gz'),
                            op_string = "-V").run()
    
    #Calculate number of cerebullum voxels (volume)
    crb_left_vox = ImageStats(in_file = join(datapath, 'sub-'+subject, 'mri','left_cerebellum.nii.gz'),
                           op_string = "-V").run()
    crb_right_vox = ImageStats(in_file = join(datapath, 'sub-'+subject, 'mri','right_cerebellum.nii.gz'),
                            op_string = "-V").run()
    
    #Calculate ICV
    intracranial_vol = ImageStats(in_file = join(datapath, 'sub-'+subject, 'mri','aparc.a2009s+aseg.nii.gz'),
                            op_string = "-V").run()
    
    #Conver to string
    hipp_left_out = str(hipp_left_vox.outputs).replace('\nout_stat = [','').split(',', 1)[0]
    hipp_right_out = str(hipp_right_vox.outputs).replace('\nout_stat = [','').split(',', 1)[0]
    amyg_left_out = str(amyg_left_vox.outputs).replace('\nout_stat = [','').split(',', 1)[0]
    amyg_right_out = str(amyg_right_vox.outputs).replace('\nout_stat = [','').split(',', 1)[0]
    crb_left_out = str(crb_left_vox.outputs).replace('\nout_stat = [','').split(',', 1)[0]
    crb_right_out = str(crb_right_vox.outputs).replace('\nout_stat = [','').split(',', 1)[0]
    icv_out = str(intracranial_vol.outputs).replace('\nout_stat = [','').split(',', 1)[0]

    #Append to data frame
    data = [subject, hipp_left_out, hipp_right_out, amyg_left_out, amyg_right_out, crb_left_out,
            crb_right_out, icv_out]
    hipp_volumes.append(data)
    return print('Completed {}'.format(subject))

In [5]:
errors = []
for sub in subject_list:
    try:
        get_volume(sub, dataV1)
    except:
        errors.append('Could not run {}'.format(sub))

200320-14:46:50,893 nipype.interface INFO:
	 stdout 2020-03-20T14:46:50.892931:mri_convert.bin --out_type nii --input_volume /Users/lucindasisk/Desktop/Milgram/candlab/data/mri/space_freesurfer/V1/sub-6108/mri/aparc.a2009s+aseg.mgz --output_volume /Users/lucindasisk/Desktop/Milgram/candlab/data/mri/space_freesurfer/V1/sub-6108/mri/aparc.a2009s+aseg.nii.gz 
200320-14:46:53,591 nipype.interface INFO:
	 stdout 2020-03-20T14:46:53.591648:$Id: mri_convert.c,v 1.226 2016/02/26 16:15:24 mreuter Exp $
200320-14:46:53,592 nipype.interface INFO:
	 stdout 2020-03-20T14:46:53.591648:reading from /Users/lucindasisk/Desktop/Milgram/candlab/data/mri/space_freesurfer/V1/sub-6108/mri/aparc.a2009s+aseg.mgz...
200320-14:46:53,593 nipype.interface INFO:
	 stdout 2020-03-20T14:46:53.591648:TR=2500.00, TE=0.00, TI=0.00, flip angle=0.00
200320-14:46:53,593 nipype.interface INFO:
	 stdout 2020-03-20T14:46:53.591648:i_ras = (-1, 0, 0)
200320-14:46:53,594 nipype.interface INFO:
	 stdout 2020-03-20T14:46:53.5916

In [9]:
voxdf = pd.DataFrame(hipp_volumes, columns = ['subjectid' , 'l_hipp_voxels', 'r_hipp_voxels' , 
                                              'l_amyg_voxels', 'r_amyg_voxels', 
                                              'l_cerebellum_voxels', 'r_cerebellum_voxels', 
                                              'icv_voxels']).sort_values(by=['subjectid'])   
voxdf.to_csv(dest + '/SPACE_hippocampal_volumes_'+today+'.csv', index=False)
voxdf.to_csv(homedest + '/SPACE_hippocampal_volumes_'+today+'.csv', index=False)